# Purpose of the script

This script performs automated text classification by matching control account titles from an Excel file to cost code descriptions stored in a SQLite database using TF-IDF vectorization and cosine similarity. It then evaluates the accuracy of the predicted categories against manually labeled true categories to support validation of the classification logic. The output is saved back to the Excel file for further analysis and reporting.

In [ ]:
import pandas as pd
import sqlite3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load Excel data
file_path = 'your_excel_file.xlsx'
sheet_name = 'Control Accounts'
df_source = pd.read_excel(file_path, sheet_name)
df_source.columns = df_source.iloc[1]
df_source = df_source[2:]
df_source = df_source[['Title']]
df_source.rename(columns={'Title': 'text'}, inplace=True)

# Load SQLite data
def fetch_table_data(table_name, db_path):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(f'SELECT * FROM {table_name}', conn)
    conn.close()
    return df

db_path = 'your_database.db'
df_categories = fetch_table_data('Cost_Code_Table', db_path)
df_categories = df_categories[['Description']]
df_categories.rename(columns={'Description': 'category'}, inplace=True)

# TF-IDF vectorization and similarity matching
all_text = pd.concat([df_source['text'], df_categories['category']], ignore_index=True)
vectorizer = TfidfVectorizer().fit(all_text)
source_vecs = vectorizer.transform(df_source['text'])
category_vecs = vectorizer.transform(df_categories['category'])

similarities = cosine_similarity(source_vecs, category_vecs)
best_matches = similarities.argmax(axis=1)
df_source['matched_category'] = df_categories['category'].iloc[best_matches].values

# Export to Excel
output_file = 'your_output_file.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_source.to_excel(writer, sheet_name='Summary', index=False)

## Test accuracy

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

# Load true categories from Excel
file_path = 'your_excel_file.xlsx'
sheet_name = 'Summary2'
df_true_category = pd.read_excel(file_path, sheet_name)
df_true_category = df_true_category[['text', 'true_category']]

# Merge with predicted categories
df_source = df_source.merge(df_true_category, on='text', how='left')

# Evaluate classification performance
accuracy = accuracy_score(df_source['true_category'], df_source['matched_category'])
report = classification_report(df_source['true_category'], df_source['matched_category'])

print("Accuracy:", accuracy)
print("Detailed Report:\n", report)

## benefits

enhances revision
enhaces analysis

## Improve accuracy

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Vectorize and train
X = vectorizer.transform(df_source['text'])
y = df_source['true_category']
model = LogisticRegression().fit(X, y)

# Predict
df_source['predicted_category'] = model.predict(X)
y_true = df_source['true_category']

X = vectorizer.transform(df_source['text'])  # Reuse the same vectorizer
y_pred = model.predict(X)

cm = confusion_matrix(y_true, y_pred, labels=model.classes_)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=model.classes_, yticklabels=model.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_true, y_pred)
report = classification_report(y_true, y_pred)

print("Accuracy:", accuracy)
print("Report:\n", report)

